In [ ]:
from datetime import datetime, timedelta, timezone

SHA_TZ = timezone(
  timedelta(hours=8),
  name='Asia/Shanghai',
)
utc_now = datetime.utcnow().replace(tzinfo=timezone.utc)
beijing_now = utc_now.astimezone(SHA_TZ)
beijing_now.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
import sys, os
import json
def is_kaggle_interactive():
  if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    return os.environ['KAGGLE_KERNEL_RUN_TYPE'] == 'Interactive'
  return True

In [ ]:
!cp /kaggle/input/aslfr-model/* ./

In [ ]:
!cat ./inference_args.json

In [ ]:
len(json.load(open('./inference_args.json'))['selected_columns'])

In [ ]:
try:
  from icecream import ic
except Exception:
  !pip install /kaggle/input/tflite-runtime-nightly-2140dev20230602-whl/tflite_runtime_nightly-2.14.0.dev20230602-cp310-cp310-manylinux2014_x86_64.whl
  !pip install -q icecream --no-index --find-links=file:///kaggle/input/icecream/ 

In [ ]:
#if is_kaggle_interactive():
from icecream import ic
import tflite_runtime.interpreter as tflite
import tflite_runtime
ic(tflite_runtime.__version__)
import time
import json
import pandas as pd
from tqdm.auto import tqdm
from leven import levenshtein
# import tensorflow as tf
import numpy as np

model_dir = './'
fold = 0
total = 20
total = 50

SEL_FEATURES = json.load(
    open(f'{model_dir}/inference_args.json'))['selected_columns']
# ic(SEL_FEATURES)

def load_relevant_data_subset(pq_path):
  return pd.read_parquet(pq_path, columns=SEL_FEATURES)  #selected_columns)

with open("/kaggle/input/asl-fingerspelling/character_to_prediction_index.json",
          "r") as f:
  character_map = json.load(f)
rev_character_map = {j: i for i, j in character_map.items()}

df = pd.read_csv('/kaggle/input/aslfr-input/train.csv')
df = df[df.fold == fold]
df = df.reset_index(drop=True)

idx = 0
sample = df.loc[idx]
loaded = load_relevant_data_subset('/kaggle/input/asl-fingerspelling/' +
                                   sample['path'])
loaded = loaded[loaded.index == sample['sequence_id']].values
print(loaded.shape)
frames = loaded

interpreter = tflite.Interpreter(f'{model_dir}/model.tflite')
# interpreter = tf.lite.Interpreter(f'{model_dir}/model.tflite')
found_signatures = list(interpreter.get_signature_list().keys())

REQUIRED_SIGNATURE = 'serving_default'
REQUIRED_OUTPUT = 'outputs'
if REQUIRED_SIGNATURE not in found_signatures:
  raise KernelEvalException('Required input signature not found.')

prediction_fn = interpreter.get_signature_runner("serving_default")
output_lite = prediction_fn(inputs=frames)
prediction_str = "".join([
    rev_character_map.get(s, "")
    for s in np.argmax(output_lite[REQUIRED_OUTPUT], axis=1)
])
ic(sample['phrase'], prediction_str)

st = time.time()
cnt = 0
model_time = 0
ic(total, len(df))
total = min(len(df), total)

l = []
for i in tqdm(range(total)):
  sample = df.loc[i]
  loaded = load_relevant_data_subset('/kaggle/input/asl-fingerspelling/' +
                                     sample['path'])
  loaded = loaded[loaded.index == sample['sequence_id']].values
  # ic(loaded)
  md_st = time.time()
  output_ = prediction_fn(inputs=loaded)
  model_time += time.time() - md_st

  prediction_str = "".join([
      rev_character_map.get(s, "")
      for s in np.argmax(output_[REQUIRED_OUTPUT], axis=1)
  ])
  distance = levenshtein(sample['phrase'], prediction_str)
  phrase_true = sample['phrase']
  phrase_pred = prediction_str
  l.append({
    'sequence_id': sample['sequence_id'],
    'phrase_true': phrase_true,
    'phrase_pred': phrase_pred,
    'phrase_len_true': len(phrase_true),
    'phrase_len_pred': len(phrase_pred),
    'distance': distance,
    'score': max(len(phrase_true) - distance, 0.) / len(phrase_true),
  })

df = pd.DataFrame(l)
score = max(df['phrase_len_true'].sum() - df['distance'].sum(), 0.) / df['phrase_len_true'].sum()
display(df.head(20))
mean_time = model_time / total
print(beijing_now.strftime('%Y-%m-%d %H:%M:%S'))
print(open('./path.txt').readline().split('working/')[1].split('/ckpt')[0])
os.system('du -h ./model.tflite')
import pandas as pd
metrics = pd.read_csv('./metrics.csv').iloc[-1]
print('score:', metrics['score'])
print('score/head', metrics['score/head'])
print('tflite_score/head:', score)
print('mean_time:', mean_time)

In [ ]:
# Package the TFLite model and inference args into a ZIP file for submission
!zip submission.zip  model.tflite inference_args.json

In [ ]:
!du -h ./submission.zip